In [16]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# 1. VERİYİ YÜKLE
# Ekran görüntüne göre csv dosyası kodun hemen yanında duruyor.
# Eğer hata verirse parantez içini '../data/raw/creditcard.csv' yaparsın.
df = pd.read_csv('creditcard.csv')

print("Veri yüklendi, işlemler başlıyor...")

# 2. FEATURE ENGINEERING (Özellik Mühendisliği)
# Scaler'ları tanımla
scaler_time = StandardScaler()
scaler_amount = StandardScaler()

# Scaling işlemleri
df['Time_Scaled'] = scaler_time.fit_transform(df[['Time']])
df['Amount_Scaled'] = scaler_amount.fit_transform(df[['Amount']])

# Yeni özellikler ekle
df['Amount_Log'] = np.log1p(df['Amount'])
df['Is_Small_Amount'] = (df['Amount'] < 10).astype(int)
df['Is_Large_Amount'] = (df['Amount'] > 200).astype(int)
df['Time_Hours'] = df['Time'] / 3600
df['Hour'] = (df['Time_Hours'] % 24).astype(int)
df['Is_Night'] = ((df['Hour'] >= 22) | (df['Hour'] <= 6)).astype(int)
df['V17_V14'] = df['V17'] * df['V14']
df['Top5_sum'] = df['V17'] + df['V14'] + df['V12'] + df['V10'] + df['V16']

# Eğitimde kullanılacak sütunları seç
features = [
    'Time_Scaled', 'Amount_Scaled',
    'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
    'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
    'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28',
    'Amount_Log', 'Is_Small_Amount', 'Is_Large_Amount',
    'Hour', 'Is_Night', 'V17_V14', 'Top5_sum'
]

X = df[features]
y = df['Class']

# 3. MODEL EĞİTİMİ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Model eğitiliyor (biraz sürebilir)...")
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

# 4. KAYDETME İŞLEMİ (Burayı düzelttik)
# Kaydedilecek klasör adı
save_path = 'models'

# Klasör yoksa oluştur (Hata almamak için)
os.makedirs(save_path, exist_ok=True)

# Modeli kaydet
with open(os.path.join(save_path, 'fraud_model.pkl'), 'wb') as f:
    pickle.dump(model, f)

# Scaler'ları kaydet
with open(os.path.join(save_path, 'scaler_time.pkl'), 'wb') as f:
    pickle.dump(scaler_time, f)

with open(os.path.join(save_path, 'scaler_amount.pkl'), 'wb') as f:
    pickle.dump(scaler_amount, f)

print("-" * 30)
print("İŞLEM TAMAMLANDI!")
print(f"Dosyalar '{save_path}' klasörüne kaydedildi:")
print("1. fraud_model.pkl")
print("2. scaler_time.pkl")
print("3. scaler_amount.pkl")

Veri yüklendi, işlemler başlıyor...
Model eğitiliyor (biraz sürebilir)...
------------------------------
✅ İŞLEM TAMAMLANDI!
Dosyalar 'models' klasörüne kaydedildi:
1. fraud_model.pkl
2. scaler_time.pkl
3. scaler_amount.pkl
